# San Joaquin  Valley Groundwater data

Related links:

* For the documentation about this dataset, its source, how to download, and the features of interest, please refer to our [Groundwater Reports Dataset](/doc/assets/groundwater.md) documentation.

*  For the explanations on how the sortage mapping datasets are mapped to TownshipRange please refer to our [Public Land Survey System](../assets/plss_sanjoaquin_riverbasin.md) documentation.

* [Data location](/work/assets/groundwater.csv)

In [11]:
import sys
sys.path.append('..')

In [12]:
import geopandas as gpd
from lib.groundwater import GroundwaterDataset

##### Groundwater measurements
https://data.cnra.ca.gov/dataset/periodic-groundwater-level-measurements

- The elevation of the groundwater surface  can be calculated by subtracting the depth to groundwater from the ground surface elevation.

> [Water levels in many aquifers](https://pubs.usgs.gov/circ/circ1217/pdf/circ1217_final.pdf) in the United States follow a natural cyclic pattern of seasonal 
> fluctuation,typically rising during the winter and spring due to greater precipitation and recharge, then declining during the summer
> and fall owing to less recharge and greater evapotranspiration. Hence, below we take only spring measurements for groundwater

In [13]:
groundwater_dataset =  GroundwaterDataset()

Pre-process the groundwater data and geospatial location and combined them into the the geospatial map_df dataset

In [14]:
groundwater_dataset.preprocess_data_df()
groundwater_dataset.preprocess_map_df()
groundwater_dataset.merge_map_with_data()

Overlay the San Joaquin Valley boundaries on the soil dataset to keep only the data in the San Joaquin Valley

In [15]:
groundwater_dataset.keep_only_sjv_data()

In [16]:
groundwater_dataset.map_df

,COUNTY_NAME,geometry,GSE_GWE,YEAR
0,Kern,POINT (-118.83700 34.97010),159.9,2017.0
1,Kern,POINT (-118.83700 34.97010),160.6,2018.0
2,Kern,POINT (-118.81500 35.00190),NaN,NaN
3,Kern,POINT (-118.81500 35.00250),1.0,1967.0
4,Kern,POINT (-118.81500 35.00250),2.2,1966.0
...,...,...,...,...
335051,Sacramento,POINT (-121.14300 38.46000),111.8,2002.0
335052,Sacramento,POINT (-121.14300 38.46000),107.0,2003.0
335053,Sacramento,POINT (-121.14300 38.46000),107.3,2004.0
335054,Sacramento,POINT (-121.14300 38.46000),110.9,2006.0


Combine all the groundwater measurement points per township, applying a _mean_ aggregation function by default  on the `GSE_GWE` feature per township

In [17]:
groundwater_dataset.aggregate_points_by_townships(feature_name="GSE_GWE")

In [18]:
groundwater_dataset.output_df

,COUNTY_NAME,YEAR,TOWNSHIP,GSE_GWE
0,Alameda,1956.0,T02S R04E,6.490000
1,Alameda,1959.0,T02S R04E,5.190000
2,Alameda,1960.0,T02S R04E,5.890000
3,Alameda,1961.0,T02S R04E,21.800000
4,Alameda,1963.0,T02S R04E,19.900000
...,...,...,...,...
28122,Tulare,2021.0,T24S R25E,228.300000
28123,Tulare,2021.0,T24S R26E,307.400000
28124,Tulare,2021.0,T24S R27E,445.907619
28125,Tulare,2021.0,T25S R26E,299.550000


In [19]:
groundwater_dataset.output_dataset_to_csv(output_filename="../assets/outputs/spring_groundwater_levels.csv")

In [20]:
groundwater_dataset.draw_mising_data_chart()

alt.LayerChart(...)